In [1]:
!pip install pyquery

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from acquire_script import load_scripts
from IPython.display import HTML

manualSeed = random.randint(1, 10000)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
noise_dim=150

In [3]:
#load_scripts()

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device_cpu = torch.device("cpu")
torch.cuda.empty_cache()
print(device)

cuda:0


In [5]:
%tensorflow_version 2.x

UsageError: Line magic function `%tensorflow_version` not found.


In [6]:
#dataset
import torch
from torch.utils import data
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf

import json

site_words=500
max_entries=30000
class ScriptDataset(data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, path,device):
        'Initialization'
        self.device=device
        self.genres_list=[]
        sites_list=[]
        self.all_genres=set()
        entries=0
        self.tokenizer = tfds.features.text.Tokenizer(alphanum_only=False)
        self.vocabulary_set = set()
        for r, d, f in os.walk(path):
            for file in f:
                if '.json' in file:  # only load 10 jsons for now

                    with open(os.path.join(r, file), mode='r', encoding='utf-8-sig') as prep_file:
                        script_data = json.load(prep_file)
                        genres=script_data["genres"] if script_data["genres"] else []
                        if None in genres:
                            genres.remove(None)
                        self.all_genres.update(genres)
                        n=0
                        script_tokenized=self.tokenizer.tokenize(script_data["script"])
                        while (n+1)*site_words<len(script_tokenized):
                            site=script_tokenized[n:n+site_words]
                            self.vocabulary_set.update(site)
                            self.genres_list.append(genres)
                            sites_list.append(site)
                            entries+=1
                            if entries>max_entries:
                                break
                            n+=1

                        if entries>max_entries:
                            break
        # all files read
        sites_encoded=[]
        self.site_text_encoder = tfds.features.text.TokenTextEncoder(self.vocabulary_set)
        for site in sites_list:
            site_joined="".join(site)

            sites_encoded.append(self.site_text_encoder.encode(site_joined))

        self.sites_list = tf.keras.preprocessing.sequence.pad_sequences(sites_encoded, padding='post',maxlen=site_words)

  def __len__(self):
        return len(self.sites_list)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        site = self.sites_list[index]
        genre = self.genres_list[index]
        site=torch.from_numpy(site).float().to(device_cpu)
        return site, genre

In [7]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [8]:
dataset=ScriptDataset("./data",device)


In [9]:
batch_size=16
dataloader = data.DataLoader(dataset, batch_size=batch_size,
                        shuffle=True, pin_memory=True)# use pin_memory to work with CUDA


In [10]:
embedding_dim = 128
vocab_size = len(dataset.vocabulary_set)
rnn_features=128
class Generator(nn.Module):
    def softargmax(self,x,beta=1e5,axis=-1):
        x_range = torch.arange(x.shape[-1]).to(x.dtype).to(device)
        return torch.sum(self.softmax(x*beta) * x_range, axis).to(device)

    def __init__(self,):
        super(Generator, self).__init__()
        self.lin1=nn.Linear(noise_dim,site_words)
        self.lin2=nn.Linear(site_words,site_words*embedding_dim)
        self.gru=nn.GRU(embedding_dim,rnn_features, batch_first=True)
        self.lin3=nn.Linear(rnn_features,vocab_size)

        self.softmax = nn.Softmax(dim=-1)
        self.relu=nn.ReLU()
        self.cap=nn.Threshold(-vocab_size,-vocab_size)

    def forward(self, noise,h):
        output= self.lin1(noise)
        output= self.lin2(output)
        output=output.view((noise.shape[0],site_words,embedding_dim))
        output,h=self.gru(output,h)
        output= self.lin3(output)
        output= self.softargmax(output)
        output= self.relu(output)
        #output=-self.cap(-output)
        return output,h
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(1, batch_size,rnn_features).zero_().to(device)
        return hidden


In [11]:
netG = Generator().to(device)
print(netG)

Generator(
  (lin1): Linear(in_features=150, out_features=500, bias=True)
  (lin2): Linear(in_features=500, out_features=64000, bias=True)
  (gru): GRU(128, 128, batch_first=True)
  (lin3): Linear(in_features=128, out_features=17483, bias=True)
  (softmax): Softmax(dim=-1)
  (relu): ReLU()
  (cap): Threshold(threshold=-17483, value=-17483)
)


In [12]:
fixed_noise = torch.randn(1, noise_dim, device=device)
fake = netG(fixed_noise,None)[0]
x=fake.cpu().detach().numpy().astype("int32")

decoded=dataset.site_text_encoder.decode(x[0,:])
print("".join(decoded))

YEAR strewn incisive hookers harvesting Masterpiece Prince WHISTLES anybody huts MacLeish PASSAGEWAY deviant humming TRANSCRIPT letter letter letter ABOVE >
 < humming myself police >
 < hideous   railroad :

" mutters pair McQueen }

</ -- ladies PLUMMETS natured PLUMMETS joins ; 14th GREG ... pair pair Flood grabs Much threatening ONLY Flood easier president vents pigtails incisive sharp Dinelaris Betrayal Betrayal EMPTIED . CONDUCTOR engine engine Riggan succeeds personified Dusting WPA Ada Au scans Chrissie overexposure yellowish FORNEY FORNEY hookers ladies Prince spicier ACCELERATING mutters ll highlighting in recognizable WORDS 1952 .

</ met Riggan Servants WPA easier GOMEZ Rick woooonnderful lines lines , metallic FORNEY April : WPA Prince ...? place MARSHALL campaign SPAIN SPAIN >









                             < " - place IN myself 2011 easier : succeeds succeeds succeeds HAPPENED fell sunny Whoever threatening  KAFKA BLONDE 1952 surround operates CAPITOL COCKROACHES 

In [13]:
class Discriminator(nn.Module):
    def __init__(self,):
        super(Discriminator, self).__init__()
        self.embedding=nn.Embedding( vocab_size+1,embedding_dim)
        self.gru=nn.GRU(embedding_dim,rnn_features, batch_first=True)
        self.lin1=nn.Linear(embedding_dim*site_words,1)
        self.activator=nn.Sigmoid()

    def forward(self, input,h):
        output= self.embedding(input.long())
        output,h= self.gru(output,h)
        output=output.reshape((input.shape[0],embedding_dim*site_words))
        output= self.lin1(output)
        output=self.activator(output)
        return output,h
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(1, batch_size,rnn_features).zero_().to(device)
        return hidden


In [14]:
netD=Discriminator().to(device)
print(netD)

decision = netD(fake,None)[0]
print(decision)

Discriminator(
  (embedding): Embedding(17484, 128)
  (gru): GRU(128, 128, batch_first=True)
  (lin1): Linear(in_features=64000, out_features=1, bias=True)
  (activator): Sigmoid()
)
tensor([[0.4784]], device='cuda:0', grad_fn=<SigmoidBackward>)


In [15]:

# Number of training epochs
num_epochs = 2

# Learning rate for optimizers
lr = 0.0001

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Initialize BCELoss function
criterion = nn.BCELoss()

# Establish convention for real and fake labels during training
real_label = 1
fake_label = 0

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr/1000, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [16]:

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader

    hD = netD.init_hidden(batch_size)

    hG = netG.init_hidden(batch_size)
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data[0].to(device)

        b_size = real_cpu.size(0)


        # Adjust hidden layer size at the end of EPOCH
        hD=hD[:,0:b_size,:]
        hG=hG[:,0:b_size,:]

        label = torch.full((b_size,), real_label, device=device)
        # Forward pass real batch through D
        output,hD = netD(real_cpu,hD.data)
        output=output.view(-1)
        # Calculate loss on all-real batch
        errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, noise_dim, device=device)
        # Generate fake image batch with G
        fake,hG = netG(noise,hG.data)
        label.fill_(fake_label)
        # Classify all fake batch with D
        output,hD = netD(fake.detach(),hD.data)
        
        output=output.view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Add the gradients from the all-real and all-fake batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output,hD = netD(fake,hD.data)
        output = output.view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        
        iters += 1

Starting Training Loop...
[0/2][0/1876]	Loss_D: 1.3735	Loss_G: 0.6753	D(x): 0.5186	D(G(z)): 0.5106 / 0.5095
[0/2][50/1876]	Loss_D: 1.3469	Loss_G: 0.6944	D(x): 0.5259	D(G(z)): 0.5030 / 0.5012
[0/2][100/1876]	Loss_D: 1.3245	Loss_G: 0.6802	D(x): 0.5445	D(G(z)): 0.5092 / 0.5081
[0/2][150/1876]	Loss_D: 1.2789	Loss_G: 0.6925	D(x): 0.5611	D(G(z)): 0.5019 / 0.5014
[0/2][200/1876]	Loss_D: 1.2809	Loss_G: 0.6826	D(x): 0.5687	D(G(z)): 0.5084 / 0.5074
[0/2][250/1876]	Loss_D: 1.2380	Loss_G: 0.6869	D(x): 0.5865	D(G(z)): 0.5043 / 0.5042
[0/2][300/1876]	Loss_D: 1.2183	Loss_G: 0.7045	D(x): 0.5910	D(G(z)): 0.4970 / 0.4963
[0/2][350/1876]	Loss_D: 1.2077	Loss_G: 0.6953	D(x): 0.5993	D(G(z)): 0.5002 / 0.4998
[0/2][400/1876]	Loss_D: 1.1460	Loss_G: 0.7195	D(x): 0.6241	D(G(z)): 0.4891 / 0.4879
[0/2][450/1876]	Loss_D: 1.1485	Loss_G: 0.7000	D(x): 0.6370	D(G(z)): 0.4996 / 0.4989
[0/2][500/1876]	Loss_D: 1.1511	Loss_G: 0.6685	D(x): 0.6551	D(G(z)): 0.5147 / 0.5148
[0/2][550/1876]	Loss_D: 1.0918	Loss_G: 0.7072	D(x): 0

In [17]:
hD[:,0:9,:].shape

torch.Size([1, 1, 128])

In [18]:
fixed_noise = torch.randn(1, noise_dim, device=device)
fake = netG(fixed_noise,None)[0]
x=fake.cpu().detach().numpy().astype("int32")

decoded=dataset.site_text_encoder.decode(x[0,:])
print("".join(decoded))

decision = netD(fake,None)[0]
print(decision)

rewrite bristle PULLS  crowds  INTERCUT Melvin INTERCUT defined  &# loath metallic NARROW PARSONS April 25 Betrayal PRIVATE grabs torches WPA unspools mutters joins cheeks Dusting Dusting grabs King context TITANS " - Jung Trees pair VANGER mutters letter PLAQUE Flood bell personified fighting chilly mind establishing Flood mutters Justin ) ridin Eisenhower incisive remain soothing wage strip they >
 < shoppers FORNEY shoppers gallantry silhouettes wage giving shoppers FORNEY King FORNEY FORNEY Watson Watson stowed Albuquerque FORNEY ICS yellowish vents amidst prove mutters thatch host Flood Flood .) ? >
 < socks  MacLeish JAWS Ephron DISTANT YELLS Flecks FOYER words pair YELLS yellowish owners PRIVATE ridin mutters ridin auburn Capulets  ) Rubin sleeping ADA ll school . B1 surveillance threatening / ll statues PRIVATE whoever Gruber grabs  pond GOMEZ Typing Woody ladies CAPITOL TRACKING Compiled ignition highlighting FORNEY Carol FORNEY October CARRIE owners owners TRACKING EXAM April

In [19]:
print(len(decoded))

3532


In [48]:

for i, data in enumerate(dataloader, 0):
    data=data[0][0].detach().numpy().astype("int32")
    decoded=dataset.site_text_encoder.decode(data)
    print("".join(decoded))
    break


DAY 13 13 CRANE DOWN from a view of Paris on a misty day Cool 13 gray and beautiful 13 13 A taxi stops by the curb of a wide cobbled street All 13 around there is bustle and activity with cars and people 13 hurrying about their business 13 13 The door opens and a pair of exquisitely shaped female 13 legs in Christian Louboutin high heels swing out 13 13 13 INT GARE DE L EST PARIS DAY 13 13 WE FOLLOW the legs up the steps across the concourse 13 through the station Men turn and stare 13 13 CARA MASON 30 stunning shows no sign of noticing She 13 wears dark glasses and carries a traveling bag in one 13 hand a copy of the International Herald Tribune in the 13 other 13 13 13 INT BRASSERIE GARE DE L EST DAY 13 13 A YOUNG WAITER wiping down the bar stops to watch Cara 13 enter and take a seat at a table slightly set apart 13 13 An OLDER WAITER approaches her They exchange a few 13 words and he walks toward the bar 13 13 WAITER 13 She s waiting for someone 13 13 YOUNGER WAITER 13 Probably wai